# Root detection

In this notebook I will test different masking techniques to make image segmentation so that roots can be separated from the background.

### Reset notebook

In [1]:
#%reset

### Import libraries

In [2]:
from matplotlib import pyplot as plt
from skimage import exposure
import numpy as np
import cv2

### Read image, resize and turn to gray scale

In [3]:
def scale_crop_gray(im):
    # Set image size to 2338x1653 (Scale = 0.5). Orig size 4676 × 3306.
    # We must use fixed size, because we can not know size of 
    # the pics taken in the future and size is used in parameter tuning.
    width = int(2338)
    height = int(1653)
    dim = (width, height)
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    
    # Cut out 50 border pixels from each side
    im = im[50:, 50:]
    im = im[:-50, :-50]
    
    # Convert the image to grayscale
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    # Return color and grayscale images
    return im, im_gray

### Read images

In [4]:
# Read color image
#image = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.09.15_033029_060_DYY.jpg', 1)

# Hyde scanner1 example
#im1 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.16_033029_030_DYY.jpg', 1)
#im2 = cv2.imread('data/Hyde_scanner1/hydescan1_T001_L001_2018.08.23_033029_037_DYY.jpg', 1)

# Varrio_Scanner2 example
im1 = cv2.imread('data/Varrio_Scanner2/VS2_T001_L001_2019.07.24_090749_132_DYY.jpg', 1)
im2 = cv2.imread('data/Varrio_Scanner2/VS2_T001_L001_2019.07.25_091204_133_DYY.jpg', 1)

# Get rescaled and cropped images in color and gray
im1, im1_gray = scale_crop_gray(im1)
im2, im2_gray = scale_crop_gray(im2)

### Make light areas mask

Filtering based on color

Input: one grayscale image

In [5]:
def light_areas_mask(im_gray):
    # Remove noise with gaussian blur (3, 3 works)
    im_blur = cv2.GaussianBlur(im_gray,(3,3),0)

    # Edges to binary (black and white) (180 and 250)
    ret, im_bin = cv2.threshold(im_blur,180,256,cv2.THRESH_BINARY)

    # Remove isolated pixels
    im_blur = cv2.GaussianBlur(im_bin,(5,5),0)
    im_bin = cv2.threshold(im_blur, 150 , 250, cv2.THRESH_BINARY)
    im_bin = im_bin[1]

    # Remove isolated pixels
    # "Make areas thinner"
    im_bin = cv2.erode(im_bin, None, iterations=1)
    # "Make areas fatter"
    im_bin = cv2.dilate(im_bin, None, iterations=1)

    # Return results
    return im_bin

### Make edge areas mask

Filtering based on edges

Input: one grayscale image

In [6]:
def edge_areas_mask(im_gray):
    #thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
    #                               cv2.THRESH_BINARY_INV, 11, 7)
    
    # Blur with bilateralFilter. BilateralFilter does not blur edges.
    blur = cv2.bilateralFilter(im_gray, 50, 15, 10)
    
    # Remove noise with gaussian blur
    blur = cv2.GaussianBlur(blur,(5,5),0)
    
    # Find edges with canny edge detection
    edges = cv2.Canny(blur, 50, 250)
    
    # "Make areas fatter" to fill missing pixels in root edges
    edges = cv2.dilate(edges, None, iterations=2)
    
    # Get contrours
    cnts, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over our contours. Remove controus with area less than xxx.
    contours=[]
    for c in cnts:
        size = cv2.contourArea(c)
        if size > 20:
            contours.append(c)
    
    # Make empty picture
    empty = np.zeros(im_gray.shape, dtype = "uint8")
    
    # Save contours to empty image
    contours = cv2.drawContours(empty, contours, -1, 255, -1)
    
    # "Make areas fatter" to fill missing pixels in root edges
    contours = cv2.dilate(contours, None, iterations=2)
    
    #Return contour areas
    return contours

### Make mask by minusing pixel values in two images

Filtering based on difference between two images

Input: two grayscale images

In [7]:
def change_areas_mask(im1_gray, im2_gray):
    # Remove noise with gaussian blur
    im1_gray = cv2.GaussianBlur(im1_gray,(25,25),0)
    im2_gray = cv2.GaussianBlur(im2_gray,(25,25),0)
    
    # Calculate change
    change = im2_gray-im1_gray
    
    # Threshold and turn to binary
    ret, change = cv2.threshold(change,40,256,cv2.THRESH_BINARY)
    
    # Remove isolated pixels
    # "Make areas thinner"
    #change = cv2.erode(change, None, iterations=5)
    # "Make areas fatter"
    #change = cv2.dilate(change, None, iterations=5)
    
    # Return results
    return change

### Add text to image

In [8]:
def add_text(im, text):
    # Set text font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Add text
    cv2.putText(img=im, text=text, org=(50,1500), fontFace=font, fontScale=3, 
                color=(255, 255, 255), thickness=3, lineType=cv2.LINE_AA)
    
    # Return result
    return im

### Print light and edge areas as binary masks

In [9]:
# Get areas
light_areas = light_areas_mask(im2_gray)
edge_areas = edge_areas_mask(im2_gray)
change_areas = change_areas_mask(im1_gray, im2_gray)

# Make supermask. Combine all masks.
combined_areas = light_areas & edge_areas & change_areas

# Concat gray, light_areas, edge_areas and change_areas to one image
concat1 = np.concatenate((im1_gray, light_areas), axis=0)
concat2 = np.concatenate((edge_areas, change_areas), axis=0)
concat3 = np.concatenate((concat1, concat2), axis=0)
concat4 = np.concatenate((concat3, combined_areas), axis=0)

# Save image
cv2.imwrite('Detect_roots_V2.png', concat4)

True

### Print images with colors

In [10]:
# Mask orig color image with mask (Add colors to images)
light_areas = cv2.bitwise_or(im2, im2, mask=light_areas)
edge_areas = cv2.bitwise_or(im2, im2, mask=edge_areas)
change_areas = cv2.bitwise_or(im2, im2, mask=change_areas)
combined_areas = cv2.bitwise_or(im2, im2, mask=combined_areas)

# Add names to images
im2 = add_text(im2, "Original image")
light_areas = add_text(light_areas, "Color filter")
edge_areas = add_text(edge_areas, "Edge filter")
change_areas = add_text(change_areas, "Change filter (pic2-pic1)")
combined_areas = add_text(combined_areas, "All filters combined")

# Concat original, light_areas, edge_areas and change_areas to one image
concat1 = np.concatenate((im2, light_areas), axis=0)
concat2 = np.concatenate((edge_areas, change_areas), axis=0)
concat3 = np.concatenate((concat1, concat2), axis=0)
concat4 = np.concatenate((concat3, combined_areas), axis=0)

# Show image
#cv2.imshow("imgages", vertical_concat)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Save images grayscale
cv2.imwrite('Detect_roots_V2_color.png', concat4)

True